#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 3:   Collaborative Filtering + Matrix Factorization

### 100 points [6% of your final grade]

### Due: Wednesday, March 27, 2019 by 11:59pm

*Goals of this homework:* Understand how collaborative filtering and matrix factorization works. Explore different methods for real-world recommendation senarios.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Collaborative Filtering (50 points)

In this part, you will implement collaborative filtering on the [MovieLens Latest Datasets](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html). After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

As background, read [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf) and refer to the course slides `week06rec.pdf` for collaborative filtering.

## 1.1 Load the Data

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two set. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [149]:
import csv
import numpy as np
import math

In [147]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...
# use a user dictionary, key => user  value => list of movie to rating
def load_data(fileName):
    user_dict = {}
    movie_dict = {}
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = ',')
        for counter, row in enumerate(csv_reader):
            if counter > 0: 
                user_temp = [int(row[1]), int(row[2])]
                movie_temp = [int(row[0]), int(row[2])]
                if int(row[0]) not in user_dict:
                    user_dict[int(row[0])] = []
                if int(row[1]) not in movie_dict:
                    movie_dict[int(row[1])] = []
                user_dict.get(int(row[0])).append(user_temp)
                movie_dict.get(int(row[1])).append(movie_temp)
    return user_dict, movie_dict, counter
            

In [125]:
def build_matrix(user_dict, movie_dict):
    res = np.zeros((len(user_dict), len(movie_dict)))
    for user, movies in user_dict.items():
        for movie in movies:
            res[int(user)][int(movie[0])] = int(movie[1])
    return res



In [126]:
USER_NUM = len(train_user_dict)
MOVIE_NUM = len(train_movie_dict)

In [148]:
train_user_dict, train_movie_dict, train_counter = load_data("train_movie.csv")
test_user_dict, test_movie_dict, test_counter = load_data("test_movie.csv")

In [128]:
train_matrix = build_matrix(train_user_dict, train_movie_dict)
test_matrix = build_matrix(test_user_dict, test_movie_dict)


In [129]:
print(len(train_user_dict), len(train_movie_dict))
print(len(test_user_dict), len(test_movie_dict))

541 1211
541 1211


In [130]:
print(train_matrix[0])
print(train_matrix[3])

s = np.dot(train_matrix[0], train_matrix[3])
print(s)
                                   

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
451.0


## 1.2 Implement User-based Collaborative Filtering

In this part, you will implement the basic User–User Collaborative Filtering algorithm introduced in the class. Given the ratings by each user, you are going to try different methods in calculating the similarities between users. You will use equation `(c)` in `week06rec.pdf` (Page 40) to aggregate ratings. Set k = 0.05. Just consider all users as neighbors. That is, while predicting how user $u$ will rate item $i$, $\widehat{C}$ includes all users who have ratings on i in the training set.

*For this memory-based algorithm, you can only rely on the ratings in training set to predict for the testing set. That is, you assume that you don't know any ratings information in the test set except that when you evalaute your model.*

### Cosine Similarity

First, you will try to calculate the similarity between users with cosine similary following the equation on page 16 of [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). And then you need to predict the rating for each (user, movie) tuple in the test set. *Note: you don't need to subtract user mean baseline from the ratings prior to computing the similarity.*

In [146]:
# Your Code Here...
# Predict for test set

def cos_similar(matrix, id_one, id_two):
    a = np.dot(matrix[id_one], matrix[id_two])
    b = sum(np.power(matrix[id_one], 2))
    c = sum(np.power(matrix[id_two], 2))
    return a / (b * c)

def predict(user, movie, matrix, train_user_dict):
    aggregate = 0
    for i in range(USER_NUM):
        if i is not user and contains(i, movie, train_user_dict):
            aggregate += (cos_similar(matrix, i, user) * (matrix[i][movie] - average(i, train_user_dict)))
    return aggregate * 0.05 
                          
def average(user, user_dict):
    ratings = user_dict.get(user)
    total = 0
    num = 0
    for rating in ratings:
        total += rating[1]
        num += 1
    return total / num

def mean_map(user_dict):
    res = {}
    for user, ratings in user_dict.items():
        total = 0
        num = 0
        for rating in ratings:
            total += rating[1]
            num += 1
        res[user] = total / num
    return res;

def contains(user, movie, user_dict):
    ratings = user_dict.get(user)
    for rating in ratings:
        if rating[0] == movie:
            return True
    return False;

In [136]:
mean_dict = mean_map(train_user_dict)

In [153]:
for user, ratings in test_user_dict.items():
    for rating in ratings:
        result = mean_dict.get(user) + predict(user, rating[0], train_matrix, train_user_dict)
        rating.append(result)
        print(rating)

print(te)



[635, 4, 4.463360750696684]
[587, 5, 4.463300125334025]
[339, 5, 4.463454292219296]
[250, 5, 4.463411259180272]
[411, 5, 4.463420286964065]
[324, 5, 4.463438173900836]
[589, 3, 4.4633656906315675]
[267, 5, 4.463664285248631]
[382, 3, 4.463437525704668]
[473, 4, 4.463366405325968]
[639, 5, 4.463422994809782]
[396, 4, 4.46333417549011]
[251, 5, 4.4634046672923935]
[569, 5, 4.463546343428454]
[480, 4, 4.463561845277641]
[203, 3, 4.463219805423005]
[629, 5, 4.463423939418722]
[663, 5, 4.463562721339072]
[577, 5, 4.463456963216286]
[621, 5, 4.4634143374333295]
[258, 4, 4.46340942023452]
[519, 4, 4.463368513991831]
[614, 4, 4.463438608746852]
[493, 5, 4.46335883068553]
[335, 5, 4.463586134310532]
[92, 3, 4.463891928555849]
[304, 2, 4.463539813693699]
[429, 5, 4.46352208446482]
[596, 5, 4.4634371077901145]
[625, 4, 4.463402192021725]
[326, 3, 4.463589200932434]
[574, 5, 4.463483677750551]
[190, 3, 4.46328036813341]
[529, 5, 4.46357539745932]
[371, 3, 4.463316059978856]
[2, 4, 4.46336389381192

[173, 4, 3.4831666502525276]
[58, 3, 3.482339108004347]
[89, 4, 3.482128202946048]
[117, 1, 3.4823588885911643]
[179, 3, 3.4827348503663647]
[43, 4, 3.4833450415383957]
[1094, 3, 3.393048007061642]
[1181, 1, 3.3928647273850956]
[1186, 4, 3.3928722026447296]
[922, 3, 3.392873982987787]
[619, 4, 3.3930637183473613]
[1195, 2, 3.392834743126698]
[1203, 4, 3.3928813555527926]
[91, 3, 3.393063551785372]
[99, 5, 3.39389464243851]
[1064, 2, 3.393164800672583]
[80, 5, 3.3934380815439367]
[0, 2, 3.3930287565118458]
[608, 3, 3.3930351043723155]
[767, 4, 3.3927085963575823]
[707, 4, 3.3928906022053993]
[717, 5, 3.393080337194503]
[811, 5, 3.3928263833954007]
[890, 3, 3.3930810346545246]
[1202, 2, 3.3928702127240853]
[491, 1, 3.3928092874383613]
[1151, 3, 3.392793278279744]
[331, 3, 3.3930218256540803]
[945, 3, 3.392805364905152]
[1183, 4, 3.3928837601460238]
[473, 4, 3.392811350305909]
[1187, 1, 3.392909737759651]
[1057, 3, 3.3928144636860864]
[850, 4, 3.3927584259939083]
[1188, 2, 3.3929014257062

[358, 3, 2.833310160410518]
[731, 2, 2.833374288919502]
[427, 3, 2.8332249772736]
[289, 5, 2.8335802780127914]
[409, 2, 2.833257100988705]
[447, 2, 2.833155364337176]
[143, 3, 2.8333281331657427]
[445, 2, 2.8332519662154887]
[171, 3, 2.8334444724105894]
[572, 2, 2.8332741008710434]
[80, 4, 2.833778228673414]
[711, 2, 2.8333147296520775]
[30, 2, 2.8332189579799834]
[154, 2, 2.833187390290327]
[325, 3, 2.8333369762057012]
[360, 3, 2.8334016388852534]
[223, 5, 2.8333897643552146]
[654, 3, 2.8332929932480058]
[147, 2, 2.8333312433321995]
[737, 1, 2.833243224384991]
[591, 3, 2.8331983105508356]
[662, 2, 2.833415292791216]
[569, 2, 2.8334752607379428]
[726, 2, 2.8333061001895006]
[438, 2, 2.833206694589608]
[174, 5, 2.8332109251276436]
[191, 3, 2.833308745950911]
[669, 1, 2.8333111978389622]
[427, 3, 3.4910012385278155]
[850, 4, 3.4910495553259326]
[248, 4, 3.4911934176107042]
[597, 1, 3.4908995738636994]
[475, 4, 3.4910421959146505]
[713, 1, 3.491066643810584]
[738, 3, 3.491086750348329]
[7

[688, 4, 2.8138491827220458]
[911, 3, 2.8138371025008473]
[875, 3, 2.8139054742236262]
[619, 4, 2.814098893624003]
[67, 3, 2.8138681173084126]
[1029, 3, 2.8139552198629336]
[520, 4, 2.8138365941216157]
[496, 4, 2.8137863112712815]
[4, 3, 2.8140094116851926]
[175, 2, 2.814313889188303]
[987, 3, 2.813830316565378]
[656, 4, 2.813871674358127]
[892, 3, 2.8138930734436047]
[194, 1, 2.8138054734115343]
[407, 2, 2.813790161287879]
[966, 4, 2.813880192350182]
[949, 3, 2.813879442829428]
[302, 3, 2.8140680304429013]
[25, 3, 2.814307564493317]
[1084, 3, 2.8138674221861293]
[217, 4, 2.814276942442265]
[912, 3, 2.8138298732291775]
[827, 3, 2.814185671134102]
[203, 3, 2.8135924807282744]
[109, 3, 2.813835174499392]
[995, 4, 2.8139356336981622]
[712, 4, 2.8138744569724268]
[736, 2, 2.813789414726937]
[567, 4, 2.813812875707615]
[180, 3, 2.814154508837824]
[709, 3, 2.813828761939105]
[299, 4, 2.814193770715451]
[907, 4, 2.8140228983152014]
[853, 3, 2.8138321190303843]
[29, 3, 2.813847384327494]
[982,

[71, 1, 3.739987170800308]
[10, 2, 3.7403705147581023]
[120, 4, 3.7407079632787026]
[37, 4, 3.7415396653132387]
[65, 1, 3.7407604276384774]
[57, 3, 3.7405918606879176]
[28, 4, 3.7407368224639237]
[54, 4, 3.7404748590631254]
[36, 4, 3.740724521783506]
[90, 3, 3.7405968589776006]
[92, 5, 3.7411256121685663]
[0, 5, 3.7409980053961376]
[66, 5, 3.74092049403423]
[77, 4, 3.7407507902700274]
[173, 4, 3.741072452936788]
[181, 4, 3.7408031372628363]
[182, 3, 3.7407729965261325]
[125, 0, 3.1611978415544244]
[1090, 4, 3.1614172992452585]
[1049, 3, 3.1612594157454983]
[1087, 4, 3.161312049774574]
[457, 4, 3.161345048678522]
[944, 3, 3.161291845070191]
[202, 4, 3.1614338289969384]
[1055, 2, 3.1613192414851783]
[605, 2, 3.1611613525648607]
[1136, 3, 3.161301167944335]
[790, 3, 3.161461997208485]
[91, 4, 3.1614769476228055]
[1190, 0, 3.161312388365317]
[906, 3, 3.1613302820913223]
[499, 2, 3.1613042249292556]
[1160, 4, 3.161269076077444]
[683, 4, 3.1612490164020555]
[301, 2, 3.161549660756767]
[826, 

...

[599, 3, 4.107135481229886]
[388, 2, 4.107302839698035]
[293, 4, 4.107438353714573]
[9, 3, 4.107355166231177]
[532, 5, 4.107320356684491]
[317, 5, 4.107338782413405]
[516, 4, 4.107274754483674]
[495, 4, 4.10725528090805]
[229, 4, 4.10739308201241]
[385, 5, 4.1073537137034775]
[497, 4, 4.107310493562494]
[409, 4, 4.107254103392556]
[489, 4, 4.107333362377343]
[299, 5, 4.107529670462463]
[139, 5, 4.107503498284841]
[66, 3, 4.107394324805394]
[501, 4, 4.107295335959141]
[132, 2, 4.107168591669956]
[389, 2, 4.107235789138592]
[610, 2, 4.107266982755738]
[598, 5, 4.107240190338065]
[295, 5, 4.1074312221988025]
[202, 5, 4.107401566700633]
[166, 5, 4.107332469996516]
[75, 5, 4.107370549225056]
[318, 5, 4.107385813685588]
[221, 4, 4.107363932149112]
[432, 5, 4.107265186395908]
[451, 4, 4.107368158055726]
[265, 5, 4.107351462691273]
[424, 5, 4.1074380568184985]
[417, 5, 4.107334947845747]
[480, 5, 4.107456986448783]
[397, 1, 4.107178268411782]
[467, 2, 4.1072346089159195]
[556, 3, 4.10725865071

[210, 2, 2.717818978650359]
[1146, 3, 2.7178662211781806]
[1100, 3, 2.717903214875242]
[656, 2, 2.717881189711922]
[8, 3, 2.7179161193256274]
[886, 3, 2.7178971912345307]
[229, 3, 2.7179343020235414]
[482, 1, 2.7178191898448145]
[406, 2, 2.717786767157491]
[1128, 2, 2.717849811524717]
[1183, 3, 2.7178874624399754]
[628, 4, 2.7178926443597176]
[553, 3, 2.71782014798673]
[748, 3, 2.717930814673536]
[483, 3, 2.71768683943776]
[141, 2, 2.717857279243581]
[1115, 2, 2.717883717535777]
[46, 1, 2.7177530012461117]
[48, 3, 2.717910037884106]
[515, 2, 2.717849536415961]
[267, 5, 2.718119866048613]
[446, 2, 2.7178988410757277]
[892, 3, 2.7179005836835763]
[316, 4, 2.7179304186273234]
[562, 3, 2.7178352634350538]
[496, 3, 2.7178157597101724]
[745, 3, 2.717860901662459]
[268, 3, 2.7179501272746323]
[336, 2, 2.7178666136512515]
[755, 2, 2.7178508021133583]
[938, 3, 2.846819817690892]
[109, 2, 2.8468375124970002]
[802, 2, 2.846742424094546]
[793, 3, 2.846828665672576]
[249, 3, 2.8468195799637024]
[70

[678, 2, 3.34939861537602]
[586, 2, 3.349460532873624]
[33, 4, 3.349475281554967]
[0, 4, 3.3495446158257938]
[318, 3, 3.349520516369284]
[62, 4, 3.349414550420883]
[278, 4, 3.349434820498563]
[283, 5, 3.349431790990677]
[191, 3, 3.349408783033747]
[19, 4, 3.349453369843315]
[75, 4, 3.349500103314605]
[304, 5, 3.349574632484942]
[259, 3, 3.349441545757604]
[4, 4, 3.3495588237273717]
[669, 4, 3.349431376706855]
[522, 5, 3.349481704209304]
[305, 3, 3.349474631182645]
[619, 5, 3.3496809586752154]
[441, 3, 3.3494052966920025]
[331, 5, 3.3495618384214345]
[622, 3, 3.3494564552140567]
[692, 3, 3.3494418677859876]
[712, 5, 3.349436484826346]
[392, 3, 3.3493800079002343]
[341, 5, 3.3494946980869535]
[338, 1, 3.3494850000613443]
[629, 3, 3.3494529162432847]
[452, 5, 3.3494397483190257]
[618, 1, 3.349408373105834]
[773, 4, 3.3495014047965563]
[238, 5, 3.3494811797795214]
[512, 4, 3.349543842274828]
[152, 3, 3.349446750372891]
[271, 2, 3.349415273332459]
[265, 4, 3.349477790364956]
[344, 4, 3.3494

[237, 3, 3.0773535215154277]
[17, 3, 3.077514580833222]
[161, 3, 3.0772540989981443]
[71, 3, 3.077037208844255]
[489, 0, 3.0773177412176866]
[700, 3, 3.0773188744795643]
[490, 1, 3.0772625306734263]
[676, 2, 3.07729397796427]
[730, 3, 3.0772120950546213]
[1050, 4, 3.077331889129464]
[25, 4, 3.0776559918304094]
[778, 4, 3.077215331273032]
[827, 4, 3.0775517217703694]
[124, 3, 3.0772229500602375]
[165, 3, 3.0773171894021045]
[671, 3, 3.0772854282577806]
[133, 0, 3.077104566839467]
[38, 3, 3.0773973365951046]
[883, 4, 3.0772347685231707]
[646, 3, 3.077311279195299]
[246, 3, 3.0773100710530144]
[663, 4, 3.0774791313922036]
[401, 3, 3.0771946994486545]
[393, 3, 3.07730198002458]
[104, 2, 3.077294649888971]
[880, 4, 3.077219235587471]
[426, 2, 3.077121758074796]
[699, 2, 3.077294459701766]
[867, 3, 3.077225298358563]
[936, 4, 3.077300856592425]
[925, 4, 3.0775487569338384]
[643, 3, 3.0772462727067045]
[90, 2, 3.07726578311198]
[306, 4, 3.077475477277795]
[835, 4, 3.0772620955244627]
[299, 4,

[1087, 4, 3.7202359130219302]
[1126, 3, 3.720174533200166]
[279, 5, 3.720427520838727]
[956, 4, 3.7202504123201487]
[1144, 4, 3.7203084967780975]
[864, 4, 3.720189196428053]
[64, 5, 3.720227381662475]
[99, 3, 3.7207854973035435]
[1157, 5, 3.7201849478061484]
[1132, 4, 3.720227910258005]
[1201, 5, 3.720195687656722]
[454, 5, 3.720244675910102]
[723, 5, 3.7202629220304484]
[310, 5, 3.7202676652513325]
[426, 3, 3.720079117718562]
[872, 5, 3.7202307410805027]
[850, 4, 3.72018408988634]
[277, 4, 3.7201921489840224]
[278, 4, 3.7202235631347405]
[970, 4, 3.720227754695932]
[1183, 4, 3.720238737036143]
[1136, 5, 3.7202360398046914]


In [158]:
print(test_user_dict.get(1))

[[1100, 3, 3.499944163668927], [1185, 4, 3.500059889901196], [424, 4, 3.5002797988875094], [1195, 5, 3.499864562890045], [1113, 4, 3.5002642484936537], [1075, 5, 3.5000169991825065], [1175, 5, 3.500153331594026]]


### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [157]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test set

for user, ratings in test_user_dict.items():
    MAE = 0
    RMSE = 0
    for rating in ratings:
        MAE += abs(rating[1] - rating[2])
        RMSE += pow((rating[1] - rating[2]), 2)
    MAE /= test_counter
    RMSE = math.sqrt(RMSE / test_counter)
    
print(MAE, RMSE)

0.0055952409605116065 0.07336159722083077


### Pearson correlation

Then, you will try to calculate the similarity between users with pearson correlation following `week06rec.pdf` (Page 37). And then you need to predict the rating for each (user, movie) tuple in the test set.

In [ ]:
# Your Code Here...
# Predict for test set



### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [ ]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



### Pearson correlation (varying the threshold)

In [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf), they observe that: 

> Pearson correlation suffers from computing high similarity
between users with few ratings in common. This can be alleviated by setting a threshold on the number of co-rated items
necessary for full agreement (correlation of 1) and scaling the
similarity when the number of co-rated items falls below this
threshold.

So now revise your Pearson to consider a threshold. Try several values and report for one that you think is appropriate.

In [ ]:
# Your Code Here...
# Predict for test set



### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [ ]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



What do you observe? How different are your results for the original Pearson Correlation approach vs. the thresholded version vs. the Cosine Similarity approach? Why do you think this is? *provide a brief (1-2 paragraph) discussion based on these questions.*

# Part 2. MF (20 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set. *You are expected to get a lower MAE and RMSE compared with the results in Part 1.*

In [ ]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

# Part 3. Extension (30 points)

Given your results in the previous parts, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system, and so on. *As in the last homework, you can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than both of your CF and MF (of course we will also judge whether what you do here is reasonable or not). Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [ ]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test



Please explain what you do to improve the recommendation in 1-2 paragraphs.

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*